In [37]:
import numpy as np
import pandas as pd
import time
from copy import deepcopy
from scipy.spatial.distance import cosine
from scipy.spatial.distance import jaccard

In [17]:
X = pd.read_csv("./kmeans_data/data.csv")
Y = pd.read_csv("./kmeans_data/label.csv")
seedVal = 42

In [18]:
X.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
Y.head()

,7
0,2
1,1
2,0
3,4
4,1


In [20]:
X = X.values
Y = Y.values

In [24]:
def pairwiseDistance(distanceType, point1, point2):
    if distanceType == "E":
        return np.sqrt(np.sum((point1 - point2) ** 2))
    elif distanceType == "C":
        return cosine(point1, point2)

    return jaccard(point1, point2)

In [62]:
def kmeans(k, distanceType, stopCriteria=None, maxIter=300, threshold=0.1):
  np.random.seed(seedVal)
  initCentroids = X[np.random.choice(X.shape[0], k, replace=False)]


  iteration = 0
  pointCluster = [-1] * X.shape[0]
  clusterDistance = [np.inf] * X.shape[0]

  SSE = np.inf

  startTime = time.time()
  while True:
    for i in range(X.shape[0]):
      clusterDistance[i] = np.inf

      for j in range(k):
        currDist = pairwiseDistance(distanceType, initCentroids[j], X[i])
        if currDist < clusterDistance[i]:
          clusterDistance[i] = currDist
          pointCluster[i] = j

    currSSE = sum(clusterDistance)
    updatedCentroids = []

    for i in range(k):
      pointsOfCluster = [idx for idx, clusterIdx in enumerate(pointCluster) if clusterIdx == i]
      clusterI = X[pointsOfCluster]
      mean = np.mean(clusterI, axis=0)
      updatedCentroids.append(mean)

    print("iteration: ", iteration," SSE: ", SSE)
    iteration += 1
    if stopCriteria == "combo":

      ifCentroidNotChanged = True
      for i in range(k):
          if np.linalg.norm(initCentroids[i] - updatedCentroids[i]) > threshold:
              ifCentroidNotChanged &= False
              break
          
      if ifCentroidNotChanged:
          break
      elif SSE < currSSE:
        break
      elif iteration == maxIter:
        break
        
    elif stopCriteria == "SSE" and SSE < currSSE:
      break
    elif stopCriteria == "centroid":
      ifCentroidNotChanged = True
      for i in range(k):
          if np.linalg.norm(initCentroids[i] - updatedCentroids[i]) > threshold:
              ifCentroidNotChanged &= False
              break
      if ifCentroidNotChanged:
          break
    elif iteration == maxIter:
        break
      
    SSE = currSSE
    initCentroids = deepcopy(updatedCentroids)
      
  endTime = time.time()
  elapsed_time = endTime - startTime
  print(f"The execution took {elapsed_time} seconds.")
  print("Total iteration required:", iteration)
  print("SSE:", SSE)

  clusterLabels = [{} for i in range(k)]
  for i in range(X.shape[0]):
    currPointLabel = Y[i][0]
    if currPointLabel not in clusterLabels[pointCluster[i]]:
      clusterLabels[pointCluster[i]][currPointLabel] = 0
    clusterLabels[pointCluster[i]][currPointLabel] += 1

  accuracies = 0
  for i in range(k):
    try:
      maxLabels = max(clusterLabels[i].values())
      allDatapoints = sum(clusterLabels[i].values())

      accuracies += (maxLabels / allDatapoints)
    except:
      accuracies += 0

  print("Accuracy:", accuracies / k * 100)

In [30]:
print("Euclidean K-means")
kmeans(10, "E")
print()

print("Cosine K-means")
kmeans(10, "C")
print()

print("Jaccard k-means")
kmeans(10, "J")
print()

Euclidean K-means
Total iteration required: 300
SSE: 15637612.937506434

Cosine K-means
Total iteration required: 300
SSE: 2497.63357883237

Jaccard k-means


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Total iteration required: 300
SSE: 9999.0



In [58]:
print("Euclidean K-means")
kmeans(10, "E")
print()

print("Cosine K-means")
kmeans(10, "C")
print()

print("Jaccard k-means")
kmeans(10, "J")
print()

Euclidean K-means
Total iteration required: 300
Accuracy: 64.12575408737415

Cosine K-means
Total iteration required: 300
Accuracy: 65.14897800117112

Jaccard k-means


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Total iteration required: 300
Accuracy: 1.1351135113511353



In [64]:
print("Euclidean K-means")
kmeans(10, "E", stopCriteria='combo', maxIter=200)
print()

Euclidean K-means
iteration:  0  SSE:  inf
iteration:  1  SSE:  20551801.690031916
iteration:  2  SSE:  16233790.361983297
iteration:  3  SSE:  15994723.842652604
iteration:  4  SSE:  15903841.628239406
iteration:  5  SSE:  15855924.657654904
iteration:  6  SSE:  15821822.928348681
iteration:  7  SSE:  15797033.409507152
iteration:  8  SSE:  15778978.732570602
iteration:  9  SSE:  15765519.19980303
iteration:  10  SSE:  15754474.87017264
iteration:  11  SSE:  15744084.779421886
iteration:  12  SSE:  15732270.092183901
iteration:  13  SSE:  15719799.800987305
iteration:  14  SSE:  15707578.761271426
iteration:  15  SSE:  15695524.213592853
iteration:  16  SSE:  15684551.694025284
iteration:  17  SSE:  15675697.775782812
iteration:  18  SSE:  15669669.766893331
iteration:  19  SSE:  15666053.125089858
iteration:  20  SSE:  15663537.964773474
iteration:  21  SSE:  15661380.24946318
iteration:  22  SSE:  15659244.760631429
iteration:  23  SSE:  15657780.402527973
iteration:  24  SSE:  1565

In [63]:
print("Cosine K-means")
kmeans(10, "C", stopCriteria='combo', maxIter=200)
print()

Cosine K-means
iteration:  0  SSE:  inf
iteration:  1  SSE:  3838.671682225782
iteration:  2  SSE:  2713.475245758675
iteration:  3  SSE:  2647.173865665005
iteration:  4  SSE:  2605.121977547506
iteration:  5  SSE:  2570.752062022472
iteration:  6  SSE:  2546.765324905738
iteration:  7  SSE:  2531.2825341407156
iteration:  8  SSE:  2522.7638642650795
iteration:  9  SSE:  2516.4732770149703
iteration:  10  SSE:  2510.748842873215
iteration:  11  SSE:  2506.832791732781
iteration:  12  SSE:  2504.8518377690625
iteration:  13  SSE:  2503.6394731792047
iteration:  14  SSE:  2502.6523894046904
iteration:  15  SSE:  2501.649485542592
iteration:  16  SSE:  2500.717042294165
iteration:  17  SSE:  2499.985792358421
iteration:  18  SSE:  2499.3062007156996
iteration:  19  SSE:  2498.8030843137703
iteration:  20  SSE:  2498.424791994796
iteration:  21  SSE:  2498.164729414861
iteration:  22  SSE:  2498.025349526285
iteration:  23  SSE:  2497.8846147129734
iteration:  24  SSE:  2497.8767817483604

In [65]:
print("Jaccard K-means")
kmeans(10, "J", stopCriteria='combo', maxIter=200)
print()

Jaccard K-means
iteration:  0  SSE:  inf
iteration:  1  SSE:  9324.701267333725
The execution took 4.2532525062561035 seconds.
Total iteration required: 2
SSE: 9324.701267333725
Accuracy: 18.456146050471357



In [50]:
######performance of Euclidean k-means when max iteration = 200 is the stopping criteria#######
print("Euclidean K-means")
kmeans(10, "E", maxIter=200)
print()

Euclidean K-means
iteration:  0  SSE:  inf
iteration:  1  SSE:  20551801.690031916
iteration:  2  SSE:  16233790.361983297
iteration:  3  SSE:  15994723.842652604
iteration:  4  SSE:  15903841.628239406
iteration:  5  SSE:  15855924.657654904
iteration:  6  SSE:  15821822.928348681
iteration:  7  SSE:  15797033.409507152
iteration:  8  SSE:  15778978.732570602
iteration:  9  SSE:  15765519.19980303
iteration:  10  SSE:  15754474.87017264
iteration:  11  SSE:  15744084.779421886
iteration:  12  SSE:  15732270.092183901
iteration:  13  SSE:  15719799.800987305
iteration:  14  SSE:  15707578.761271426
iteration:  15  SSE:  15695524.213592853
iteration:  16  SSE:  15684551.694025284
iteration:  17  SSE:  15675697.775782812
iteration:  18  SSE:  15669669.766893331
iteration:  19  SSE:  15666053.125089858
iteration:  20  SSE:  15663537.964773474
iteration:  21  SSE:  15661380.24946318
iteration:  22  SSE:  15659244.760631429
iteration:  23  SSE:  15657780.402527973
iteration:  24  SSE:  1565

In [51]:
######performance of Consie k-means when max iteration = 200 is the stopping criteria#######
print("Cosine K-means")
kmeans(10, "C", maxIter=200)
print()

Cosine K-means
iteration:  0  SSE:  inf
iteration:  1  SSE:  3838.671682225782
iteration:  2  SSE:  2713.475245758675
iteration:  3  SSE:  2647.173865665005
iteration:  4  SSE:  2605.121977547506
iteration:  5  SSE:  2570.752062022472
iteration:  6  SSE:  2546.765324905738
iteration:  7  SSE:  2531.2825341407156
iteration:  8  SSE:  2522.7638642650795
iteration:  9  SSE:  2516.4732770149703
iteration:  10  SSE:  2510.748842873215
iteration:  11  SSE:  2506.832791732781
iteration:  12  SSE:  2504.8518377690625
iteration:  13  SSE:  2503.6394731792047
iteration:  14  SSE:  2502.6523894046904
iteration:  15  SSE:  2501.649485542592
iteration:  16  SSE:  2500.717042294165
iteration:  17  SSE:  2499.985792358421
iteration:  18  SSE:  2499.3062007156996
iteration:  19  SSE:  2498.8030843137703
iteration:  20  SSE:  2498.424791994796
iteration:  21  SSE:  2498.164729414861
iteration:  22  SSE:  2498.025349526285
iteration:  23  SSE:  2497.8846147129734
iteration:  24  SSE:  2497.8767817483604

In [52]:
######performance of Jaccard k-means when max iteration = 200 is the stopping criteria#######
print("Jaccard k-means")
kmeans(10, "J", maxIter=200)
print()

Jaccard k-means
iteration:  0  SSE:  inf


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


iteration:  1  SSE:  9324.701267333725
iteration:  2  SSE:  9998.765869423221
iteration:  3  SSE:  9990.640424759325
iteration:  4  SSE:  9998.784860689759
iteration:  5  SSE:  9996.23823768446
iteration:  6  SSE:  9998.800569993033
iteration:  7  SSE:  9992.980064998756
iteration:  8  SSE:  9998.697486534084
iteration:  9  SSE:  9997.988213707691
iteration:  10  SSE:  9998.624285119557
iteration:  11  SSE:  9998.135299869697
iteration:  12  SSE:  9998.711975508093
iteration:  13  SSE:  9997.409621212255
iteration:  14  SSE:  9998.68163467911
iteration:  15  SSE:  9997.41822120939
iteration:  16  SSE:  9998.794164915706
iteration:  17  SSE:  9998.61955863192
iteration:  18  SSE:  9998.771998037946
iteration:  19  SSE:  9998.752736260529
iteration:  20  SSE:  9998.826041465254
iteration:  21  SSE:  9998.471813815415
iteration:  22  SSE:  9998.886289923055
iteration:  23  SSE:  9998.369879739064
iteration:  24  SSE:  9998.805927353364
iteration:  25  SSE:  9998.700749087579
iteration:  2

In [55]:
########### comparing distance metrics when SSE increase is the stopping criteria#########
print("Euclidean K-means")
kmeans(10, "E", maxIter=200, stopCriteria='SSE')
print()

print("Cosine K-means")
kmeans(10, "C", maxIter=200,stopCriteria='SSE')
print()

print("Jaccard k-means")
kmeans(10, "J", maxIter=200, stopCriteria='SSE')
print()

Euclidean K-means
The execution took 46.447903871536255 seconds.
Total iteration required: 55
SSE: 15638236.272363372
Accuracy: 64.11047630564309

Cosine K-means
The execution took 149.14585137367249 seconds.
Total iteration required: 200
SSE: 2497.63357883237
Accuracy: 65.14897800117112

Jaccard k-means
The execution took 2.75069260597229 seconds.
Total iteration required: 2
SSE: 9324.701267333725
Accuracy: 18.456146050471357



In [56]:
#####Comparing distance metrics when centroid change is the stopping criteria#######

print("Euclidean K-means")
kmeans(10, "E", maxIter=200, stopCriteria='centroid')
print()

print("Cosine K-means")
kmeans(10, "C", maxIter=200,stopCriteria='centroid')
print()

print("Jaccard k-means")
kmeans(10, "J", maxIter=200, stopCriteria='centroid')
print()

Euclidean K-means
The execution took 47.517340660095215 seconds.
Total iteration required: 66
SSE: 15637611.015847057
Accuracy: 64.12575408737415

Cosine K-means
The execution took 25.444697618484497 seconds.
Total iteration required: 36
SSE: 2497.636428574767
Accuracy: 65.14897800117112

Jaccard k-means
The execution took 79.06504273414612 seconds.
Total iteration required: 63
SSE: 9999.0
Accuracy: 1.1351135113511353

